<a href="https://colab.research.google.com/github/jiuuu26/ML_website-fingerprint/blob/main/feature/ML_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## monitored 특징 추출

In [ ]:
import pickle

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file
print("Loading datafile...")
with open("/content/drive/MyDrive/ML_2024_2/mon_standard.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    data = pickle.load(fi)

X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
y = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]

# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.
    for sample in data[i]:
        size_seq = []
        time_seq = []
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
        X1.append(time_seq)
        X2.append(size_seq)
        y.append(label)
size = len(y)

print(f'Total samples: {size}') # Output: 19000

Loading datafile...
Total samples: 19000


In [ ]:
num_total_packets = [] # feature 1
sum_packets = [] # feature 2
num_incoming_packets = [] # feature 3
frac_incoming_packets = [] # feature 4
num_outgoing_packets=[] # feature 5
frac_outgoing_packets=[] # feature 6
cumulative_packet_size_seq = [] # feature 7
packet_size_ratios = [] # feature 8
burst_lengths = []  # feature 9
burst_seq = []  # feature 10
average_incoming_ordering=[] # feature 11
std_dev_incoming_ordering=[] # feature 12
average_outgoing_ordering=[] # feature 13
std_dev_outgoing_ordering=[] # feature14

In [ ]:
import numpy as np

for size_seq in X2:
    # 1. Total number of pacekts
    total = len(size_seq)
    num_total_packets.append(total)

    # 2. Sum of packets (absolute value gives the size regardless of direction)
    sum_pckts = sum(abs(size) for size in size_seq)
    sum_packets.append(sum_pckts)

    # 3. Number of incoming packets
    incoming = sum(1 for size in size_seq if size < 0)
    num_incoming_packets.append(incoming)

    # 4. Number of incoming packets as a fraction of the total number of packets
    frac_incoming = incoming / total if total > 0 else 0
    frac_incoming_packets.append(frac_incoming)

    # 5. Number of outgoing packets
    outgoing = sum(1 for size in size_seq if size > 0)
    num_outgoing_packets.append(outgoing)

    # 6. Number of outgoing packets as a fraction of the total number of packets
    frac_outgoing = outgoing / total if total > 0 else 0
    frac_outgoing_packets.append(frac_outgoing)


In [ ]:
import pandas as pd

data = {
    'feature 1': num_total_packets,
    'feature 2': sum_packets,
    'feature 3': num_incoming_packets,
    'feature 4': frac_incoming_packets,
    'feature 5': num_outgoing_packets,
    'feature 6': frac_outgoing_packets
}

df = pd.DataFrame(data)
print(df.head())

   feature 1  feature 2  feature 3  feature 4  feature 5  feature 6
0       1421     727552       1300   0.914849        121   0.085151
1        518     265216        438   0.845560         80   0.154440
2       1358     695296       1240   0.913108        118   0.086892
3       1446     740352       1324   0.915629        122   0.084371
4       1406     719872       1291   0.918208        115   0.081792


In [ ]:
 for size_seq in X2:
    # 7. Cumulative Packet Size Sequence
    cumulative_size = 0
    cumulative_size_seq = []
    for size in size_seq:
        cumulative_size += size  # 누적 크기 계산
        cumulative_size_seq.append(cumulative_size)
    cumulative_packet_size_seq.append(cumulative_size_seq)

    # 8. Packet Size Ratios
    outgoing_size_sum = sum(size for size in size_seq if size > 0)
    incoming_size_sum = abs(sum(size for size in size_seq if size < 0))  # 절대값 사용
    packet_size_ratio = outgoing_size_sum / incoming_size_sum if incoming_size_sum > 0 else 0
    packet_size_ratios.append(packet_size_ratio)

    # 9. Burst Lengths
    bursts = []
    current_burst = 0
    last_direction = np.sign(size_seq[0]) if size_seq else 0  # 초기 방향 설정
    for size in size_seq:
        direction = np.sign(size)

        if direction == last_direction:
            current_burst += size
        else:
            bursts.append(current_burst)
            current_burst = size          # 새 버스트 시작
            last_direction = direction

    if current_burst != 0:
        bursts.append(current_burst)  # 마지막 버스트 추가

    burst_seq.append(bursts)               # 버스트 시퀀스 저장
    burst_lengths.append(len(bursts))      # 버스트 길이 저장

In [ ]:
data = {
    'feature 7': cumulative_packet_size_seq,
    'feature 8': packet_size_ratios,
    'feature 9': burst_lengths,
    'feature 10': burst_seq,
}

df = pd.DataFrame(data)
print(df.head())

                                           feature 7  feature 8  feature 9  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.093077        156   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.182648         94   
2  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.095161        152   
3  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.092145        160   
4  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.089078        158   

                                          feature 10  
0  [-1024, 512, -512, 512, -512, 1024, -7168, 512...  
1  [-1024, 512, -512, 512, -512, 1024, -7168, 512...  
2  [-1024, 512, -512, 512, -512, 1024, -7168, 512...  
3  [-1024, 512, -512, 512, -512, 1024, -7168, 512...  
4  [-1024, 512, -512, 512, -512, 1024, -7168, 512...  


In [ ]:
for size_seq in X2:
    # 11. Average of the incoming packet ordering list
    incoming_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size < 0:
            incoming_ordering_list.append(idx)

    average_incoming_ordering.append(np.mean(incoming_ordering_list))

    # 12. Standard deviation of the incoming packet ordering list
    std_dev_incoming_ordering.append(np.std(incoming_ordering_list))

    # 13. Average of the outgoing packet ordering list
    outgoing_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size > 0:
            outgoing_ordering_list.append(idx)
    average_outgoing_ordering.append(np.mean(outgoing_ordering_list))

    # 14. Standard deviation of the outgoing packet ordering list
    std_dev_outgoing_ordering.append(np.std(outgoing_ordering_list))

In [ ]:
data = {
    'feature 11': average_incoming_ordering,
    'feature 12': std_dev_incoming_ordering,
    'feature 13': average_outgoing_ordering,
    'feature 14': std_dev_outgoing_ordering
}

df = pd.DataFrame(data)
print(df.head())

   feature 11  feature 12  feature 13  feature 14
0  704.106154  398.486335  773.322314  515.483953
1  264.406393  150.591146  226.162500  139.231951
2  668.259677  381.878642  786.110169  472.735508
3  713.503021  406.205288  820.139344  513.916038
4  694.740511  395.028889  789.608696  503.993490


In [ ]:
print(len(num_total_packets))
print(len(sum_packets))
print(len(num_incoming_packets))
print(len(frac_incoming_packets))
print(len(num_outgoing_packets))
print(len(frac_outgoing_packets))
print(len(cumulative_packet_size_seq))
print(len(packet_size_ratios))
print(len(burst_lengths))
print(len(burst_seq))
print(len(average_incoming_ordering))
print(len(std_dev_incoming_ordering))
print(len(average_outgoing_ordering))
print(len(std_dev_outgoing_ordering))

19000
19000
19000
19000
19000
19000
19000
19000
19000
19000
19000
19000
19000
19000


#### 특징 추가 15~19

In [ ]:
avg_packet_size = []  # feature 15
max_packet_size = []  # feature 16
avg_inter_arrival_time = []  # feature 17
var_packet_size = []  # feature 18
alternative_packets_per_second_sum = [] # feature 19

for time_seq, size_seq in zip(X1, X2):
    # 15. Average Packet Size
    avg_size = sum(abs(size) for size in size_seq) / len(size_seq) if size_seq else 0
    avg_packet_size.append(avg_size)

    # 16. Max Packet Size
    max_size = max(abs(size) for size in size_seq) if size_seq else 0
    max_packet_size.append(max_size)

    # 17. Packet Inter-Arrival Time (IAT)
    inter_arrival_time = np.mean(np.diff(time_seq)) if len(time_seq) > 1 else 0
    avg_inter_arrival_time.append(inter_arrival_time)

    # 18. Variance of Packet Size
    var_size = np.var([abs(size) for size in size_seq]) if size_seq else 0
    var_packet_size.append(var_size)

    # 19. Alternative packets per second
    packets_per_subset = len(size_seq) // 20 # 20 even-sized subsets
    subset_sums = []

    for i in range(0, len(size_seq), packets_per_subset):
        start_index = i
        end_index = min(i + packets_per_subset, len(size_seq))

        time_interval = time_seq[end_index - 1] - time_seq[start_index]
        time_interval = max(time_interval, 0.001) # Ensure that the time interval does not become zero.

        packets_per_second = (end_index - start_index) / time_interval
        subset_sums.append(packets_per_second)

        if (end_index == packets_per_subset*20):
            break

    alternative_packets_per_second_sum.append(sum(subset_sums))


In [ ]:
data = {
   'feature 15':avg_packet_size,
   'feature 16':max_packet_size,
   'feature 17':avg_inter_arrival_time,
   'feature 18':var_packet_size,
   'feature 19':alternative_packets_per_second_sum
}

df = pd.DataFrame(data)
print(df.head())

   feature 15  feature 16  feature 17  feature 18     feature 19
0       512.0         512    0.007141         0.0   25281.847992
1       512.0         512    0.019652         0.0   34200.937813
2       512.0         512    0.008187         0.0  107531.177098
3       512.0         512    0.009246         0.0   23241.614790
4       512.0         512    0.007573         0.0   55080.330639


In [ ]:
print(len(avg_packet_size))
print(len(max_packet_size))
print(len(avg_inter_arrival_time))
print(len(var_packet_size))
print(len(alternative_packets_per_second_sum))

19000
19000
19000
19000
19000


#### 특징 추가 20~23

In [ ]:
num_packets_per_second = [] # feature 20
mean_packets_per_second = [] # feature 21
std_packets_per_second = [] # feature 22
med_packets_per_second = [] #feature 23

import statistics

for i in range(len(X1)):
    num_packets = len(X1[i]) / X1[i][-1]
    mean_packets = statistics.mean(X1[i])
    std_packets = statistics.stdev(X1[i])
    med_packets = statistics.median(X1[i])

    num_packets_per_second.append(num_packets)
    mean_packets_per_second.append(mean_packets)
    std_packets_per_second.append(std_packets)
    med_packets_per_second.append(med_packets)

In [ ]:
data = {
   'feature 20':num_packets_per_second,
   'feature 21':mean_packets_per_second,
   'feature 22':std_packets_per_second,
   'feature 23':med_packets_per_second,
}

df = pd.DataFrame(data)
print(df.head())

   feature 20  feature 21  feature 22  feature 23
0  140.138067    4.315749    1.128739        4.09
1   50.984252    4.611815    1.371935        4.80
2  122.232223    5.488969    1.179699        5.15
3  108.233533    5.995512    1.709479        5.57
4  132.142857    4.698869    1.297546        4.31


In [ ]:
print(len(num_packets_per_second))
print(len(mean_packets_per_second))
print(len(std_packets_per_second))
print(len(med_packets_per_second))

19000
19000
19000
19000


In [ ]:
outgoing_counts=[]
burst_packet_counts=[]
first_packet_direction = [] # feature 24
num_packets_longest_burst = [] # feature 25

for size_seq in X2:

    bursts = 1 if size_seq else 0
    current_direction = size_seq[0] if size_seq else None
    current_burst_length = 1
    longest_burst = 1
    outgoing_counts = []
    burst_packet_counts = []

    # Feature 24: First Packet Direction
    first_direction = 1 if size_seq and size_seq[0] > 0 else -1 if size_seq and size_seq[0] < 0 else 0
    first_packet_direction.append(first_direction)

    for i in range(1, len(size_seq)):
        if size_seq[i] * current_direction <= 0:  # 방향이 바뀌면 새 버스트 시작
            bursts += 1
            outgoing_counts.append(sum(1 for size in size_seq[i - current_burst_length:i] if size > 0))
            burst_packet_counts.append(current_burst_length)
            longest_burst = max(longest_burst, current_burst_length)
            current_direction = size_seq[i]
            current_burst_length = 1
        else:
            current_burst_length += 1

    if size_seq:
        outgoing_counts.append(sum(1 for size in size_seq[-current_burst_length:] if size > 0))
        burst_packet_counts.append(current_burst_length)
        longest_burst = max(longest_burst, current_burst_length)

    # Feature 25: Number of Packets in the Longest Burst 저장
    num_packets_longest_burst.append(longest_burst)

In [ ]:
data = {
   'feature 24':first_packet_direction,
   'feature 25':num_packets_longest_burst
}

df = pd.DataFrame(data)
print(df.head())

   feature 24  feature 25
0          -1          66
1          -1          50
2          -1          50
3          -1          70
4          -1          60


In [ ]:
print(len(first_packet_direction))
print(len(num_packets_longest_burst))

19000
19000


#### 14개 특징 데이터

In [ ]:
mon_data_14 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'cumulative_packet_size_seq': cumulative_packet_size_seq,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'burst_seq': burst_seq,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering
}

df_14 = pd.DataFrame(mon_data_14)
print(df_14.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0               1421       727552                  1300   
1                518       265216                   438   
2               1358       695296                  1240   
3               1446       740352                  1324   
4               1406       719872                  1291   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               0.914849                   121               0.085151   
1               0.845560                    80               0.154440   
2               0.913108                   118               0.086892   
3               0.915629                   122               0.084371   
4               0.918208                   115               0.081792   

                          cumulative_packet_size_seq  packet_size_ratios  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...            0.093077   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...  

#### 19개 특징 데이터

In [ ]:
mon_data_19 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'cumulative_packet_size_seq': cumulative_packet_size_seq,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'burst_seq': burst_seq,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
    'avg_packet_size':avg_packet_size,
    'max_packet_size':max_packet_size,
    'avg_inter_arrival_time':avg_inter_arrival_time,
    'var_packet_size':var_packet_size,
    'alternative_packets_per_second_sum':alternative_packets_per_second_sum
}

df_19 = pd.DataFrame(mon_data_19)
print(df_19.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0               1421       727552                  1300   
1                518       265216                   438   
2               1358       695296                  1240   
3               1446       740352                  1324   
4               1406       719872                  1291   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               0.914849                   121               0.085151   
1               0.845560                    80               0.154440   
2               0.913108                   118               0.086892   
3               0.915629                   122               0.084371   
4               0.918208                   115               0.081792   

                          cumulative_packet_size_seq  packet_size_ratios  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...            0.093077   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...  

#### 25개 특징 데이터

In [ ]:
mon_data_25 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'cumulative_packet_size_seq': cumulative_packet_size_seq,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'burst_seq': burst_seq,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
    'avg_packet_size':avg_packet_size,
    'max_packet_size':max_packet_size,
    'avg_inter_arrival_time':avg_inter_arrival_time,
    'var_packet_size':var_packet_size,
    'alternative_packets_per_second_sum':alternative_packets_per_second_sum,
    'num_packets_per_second':num_packets_per_second,
    'mean_packets_per_second':mean_packets_per_second,
    'std_packets_per_second':std_packets_per_second,
    'med_packets_per_second':med_packets_per_second,
    'first_packet_direction':first_packet_direction,
    'num_packets_longest_burst':num_packets_longest_burst
}

df_25 = pd.DataFrame(mon_data_25)
print(df_25.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0               1421       727552                  1300   
1                518       265216                   438   
2               1358       695296                  1240   
3               1446       740352                  1324   
4               1406       719872                  1291   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               0.914849                   121               0.085151   
1               0.845560                    80               0.154440   
2               0.913108                   118               0.086892   
3               0.915629                   122               0.084371   
4               0.918208                   115               0.081792   

                          cumulative_packet_size_seq  packet_size_ratios  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...            0.093077   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...  

#### label 확인

In [ ]:
label = {'label': y}
df_label = pd.DataFrame(label)

print(len(df_label))
print(df_label.head())

19000
   label
0      0
1      0
2      0
3      0
4      0


### csv 파일 변환

In [ ]:
df_14.to_csv('/content/drive/MyDrive/ML_2024_2/mon_features14.csv', index=False)
df_19.to_csv('/content/drive/MyDrive/ML_2024_2/mon_features19.csv', index=False)
df_25.to_csv('/content/drive/MyDrive/ML_2024_2/mon_features25.csv', index=False)
df_label.to_csv('/content/drive/MyDrive/ML_2024_2/mon_label.csv', index=False)

## unmonitored 특징 추출

In [ ]:
import pickle

TOTAL_URLS = 3000  # total number in the dataset

# Load 3000 unmon pickle file
print("Loading datafile...")
with open('/content/drive/MyDrive/ML_2024_2/unmon_standard10_3000.pkl', 'rb') as f:  # Path to unmon_standard10.pkl in Colab
    x = pickle.load(f)

size = len(x)
print(f'Total samples: {size}')

X1 = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information

for i in range(TOTAL_URLS):
    size_seq = []
    time_seq = []
    for c in x[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.
    X1.append(time_seq)
    X2.append(size_seq)

y = [-1] * 3000

print(len(X1)) # Print the length of X1
print(len(y))

Loading datafile...
Total samples: 3000
3000
3000


In [ ]:
num_total_packets = [] # feature 1
sum_packets = [] # feature 2
num_incoming_packets = [] # feature 3
frac_incoming_packets = [] # feature 4
num_outgoing_packets=[] # feature 5
frac_outgoing_packets=[] # feature 6
cumulative_packet_size_seq = [] # feature 7
packet_size_ratios = [] # feature 8
burst_lengths = []  # feature 9
burst_seq = []  # feature 10
average_incoming_ordering=[] # feature 11
std_dev_incoming_ordering=[] # feature 12
average_outgoing_ordering=[] # feature 13
std_dev_outgoing_ordering=[] # feature14

In [ ]:
for size_seq in X2:
    # 1. Total number of pacekts
    total = len(size_seq)
    num_total_packets.append(total)

    # 2. Sum of packets (absolute value gives the size regardless of direction)
    sum_pckts = sum(abs(size) for size in size_seq)
    sum_packets.append(sum_pckts)

    # 3. Number of incoming packets
    incoming = sum(1 for size in size_seq if size < 0)
    num_incoming_packets.append(incoming)

    # 4. Number of incoming packets as a fraction of the total number of packets
    frac_incoming = incoming / total if total > 0 else 0
    frac_incoming_packets.append(frac_incoming)

    # 5. Number of outgoing packets
    outgoing = sum(1 for size in size_seq if size > 0)
    num_outgoing_packets.append(outgoing)

    # 6. Number of outgoing packets as a fraction of the total number of packets
    frac_outgoing = outgoing / total if total > 0 else 0
    frac_outgoing_packets.append(frac_outgoing)


In [ ]:
import pandas as pd

data = {
    'feature 1': num_total_packets,
    'feature 2': sum_packets,
    'feature 3': num_incoming_packets,
    'feature 4': frac_incoming_packets,
    'feature 5': num_outgoing_packets,
    'feature 6': frac_outgoing_packets
}

df = pd.DataFrame(data)
print(df.head())

   feature 1  feature 2  feature 3  feature 4  feature 5  feature 6
0        130      66560        101   0.776923         29   0.223077
1       9927    5082624       9189   0.925657        738   0.074343
2        359     183808        285   0.793872         74   0.206128
3       9948    5093376       9407   0.945617        541   0.054383
4       2508    1284096       2215   0.883174        293   0.116826


In [ ]:
 for size_seq in X2:
    # 7. Cumulative Packet Size Sequence
    cumulative_size = 0
    cumulative_size_seq = []
    for size in size_seq:
        cumulative_size += size  # 누적 크기 계산
        cumulative_size_seq.append(cumulative_size)
    cumulative_packet_size_seq.append(cumulative_size_seq)

    # 8. Packet Size Ratios
    outgoing_size_sum = sum(size for size in size_seq if size > 0)
    incoming_size_sum = abs(sum(size for size in size_seq if size < 0))  # 절대값 사용
    packet_size_ratio = outgoing_size_sum / incoming_size_sum if incoming_size_sum > 0 else 0
    packet_size_ratios.append(packet_size_ratio)

    # 9. Burst Lengths
    bursts = []
    current_burst = 0
    last_direction = np.sign(size_seq[0]) if size_seq else 0  # 초기 방향 설정
    for size in size_seq:
        direction = np.sign(size)

        if direction == last_direction:
            current_burst += size
        else:
            bursts.append(current_burst)
            current_burst = size          # 새 버스트 시작
            last_direction = direction

    if current_burst != 0:
        bursts.append(current_burst)  # 마지막 버스트 추가

    burst_seq.append(bursts)               # 버스트 시퀀스 저장
    burst_lengths.append(len(bursts))      # 버스트 길이 저장

In [ ]:
data = {
    'feature 7': cumulative_packet_size_seq,
    'feature 8': packet_size_ratios,
    'feature 9': burst_lengths,
    'feature 10': burst_seq,
}

df = pd.DataFrame(data)
print(df.head())

                                           feature 7  feature 8  feature 9  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.287129         35   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.080313        775   
2  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.259649         76   
3  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.057510        732   
4  [-512, -1024, -512, -1024, -512, -1024, -512, ...   0.132280        340   

                                          feature 10  
0  [-1024, 512, -512, 512, -512, 512, -1024, 1536...  
1  [-1024, 512, -512, 512, -512, 512, -1024, 1024...  
2  [-1024, 512, -512, 512, -512, 512, -1024, 1536...  
3  [-1024, 512, -512, 512, -512, 512, -1024, 512,...  
4  [-1024, 512, -512, 512, -512, 512, -25600, 512...  


In [ ]:
for size_seq in X2:
    # 11. Average of the incoming packet ordering list
    incoming_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size < 0:
            incoming_ordering_list.append(idx)

    average_incoming_ordering.append(np.mean(incoming_ordering_list))

    # 12. Standard deviation of the incoming packet ordering list
    std_dev_incoming_ordering.append(np.std(incoming_ordering_list))

    # 13. Average of the outgoing packet ordering list
    outgoing_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size > 0:
            outgoing_ordering_list.append(idx)
    average_outgoing_ordering.append(np.mean(outgoing_ordering_list))

    # 14. Standard deviation of the outgoing packet ordering list
    std_dev_outgoing_ordering.append(np.std(outgoing_ordering_list))

In [ ]:
data = {
    'feature 11': average_incoming_ordering,
    'feature 12': std_dev_incoming_ordering,
    'feature 13': average_outgoing_ordering,
    'feature 14': std_dev_outgoing_ordering
}

df = pd.DataFrame(data)
print(df.head())

    feature 11   feature 12   feature 13   feature 14
0    72.465347    36.304696    36.758621    27.053784
1  5052.770595  2845.096286  3845.246612  2885.461055
2   184.996491   100.730751   155.905405   111.145044
3  4991.069735  2855.784563  4667.994455  3120.454082
4  1256.969752   716.374241  1227.269625   778.715376


In [ ]:
avg_packet_size = []  # feature 15
max_packet_size = []  # feature 16
avg_inter_arrival_time = []  # feature 17
var_packet_size = []  # feature 18
alternative_packets_per_second_sum = [] # feature 19

for time_seq, size_seq in zip(X1, X2):
    # 15. Average Packet Size
    avg_size = sum(abs(size) for size in size_seq) / len(size_seq) if size_seq else 0
    avg_packet_size.append(avg_size)

    # 16. Max Packet Size
    max_size = max(abs(size) for size in size_seq) if size_seq else 0
    max_packet_size.append(max_size)

    # 17. Packet Inter-Arrival Time (IAT)
    inter_arrival_time = np.mean(np.diff(time_seq)) if len(time_seq) > 1 else 0
    avg_inter_arrival_time.append(inter_arrival_time)

    # 18. Variance of Packet Size
    var_size = np.var([abs(size) for size in size_seq]) if size_seq else 0
    var_packet_size.append(var_size)

    # 19. Alternative packets per second
    packets_per_subset = len(size_seq) // 20 # 20 even-sized subsets
    subset_sums = []

    for i in range(0, len(size_seq), packets_per_subset):
        start_index = i
        end_index = min(i + packets_per_subset, len(size_seq))

        time_interval = time_seq[end_index - 1] - time_seq[start_index]
        time_interval = max(time_interval, 0.001) # Ensure that the time interval does not become zero.

        packets_per_second = (end_index - start_index) / time_interval
        subset_sums.append(packets_per_second)

        if (end_index == packets_per_subset*20):
            break

    alternative_packets_per_second_sum.append(sum(subset_sums))


In [ ]:
data = {
   'feature 15':avg_packet_size,
   'feature 16':max_packet_size,
   'feature 17':avg_inter_arrival_time,
   'feature 18':var_packet_size,
   'feature 19':alternative_packets_per_second_sum
}

df = pd.DataFrame(data)
print(df.head())

   feature 15  feature 16  feature 17  feature 18    feature 19
0       512.0         512    0.043101         0.0  25309.451476
1       512.0         512    0.001197         0.0  68180.017464
2       512.0         512    0.049749         0.0   1518.989991
3       512.0         512    0.000922         0.0  62061.265205
4       512.0         512    0.021376         0.0  36068.101946


In [ ]:
num_packets_per_second = [] # feature 20
mean_packets_per_second = [] # feature 21
std_packets_per_second = [] # feature 22
med_packets_per_second = [] #feature 23

import statistics

for i in range(len(X1)):
    num_packets = len(X1[i]) / X1[i][-1]
    mean_packets = statistics.mean(X1[i])
    std_packets = statistics.stdev(X1[i])
    med_packets = statistics.median(X1[i])

    num_packets_per_second.append(num_packets)
    mean_packets_per_second.append(mean_packets)
    std_packets_per_second.append(std_packets)
    med_packets_per_second.append(med_packets)

In [ ]:
data = {
   'feature 20':num_packets_per_second,
   'feature 21':mean_packets_per_second,
   'feature 22':std_packets_per_second,
   'feature 23':med_packets_per_second,
}

df = pd.DataFrame(data)
print(df.head())

    feature 20  feature 21  feature 22  feature 23
0    23.381295    3.799462    1.309629        4.18
1   835.606061    9.884457    1.613041       10.23
2    20.157215    9.328440    3.082876        8.68
3  1084.841876    7.584566    1.036050        7.64
4    46.799776   10.352085    5.565770        8.08


In [ ]:
outgoing_counts=[]
burst_packet_counts=[]
first_packet_direction = [] # feature 24
num_packets_longest_burst = [] # feature 25

for size_seq in X2:

    bursts = 1 if size_seq else 0
    current_direction = size_seq[0] if size_seq else None
    current_burst_length = 1
    longest_burst = 1
    outgoing_counts = []
    burst_packet_counts = []

    # Feature 24: First Packet Direction
    first_direction = 1 if size_seq and size_seq[0] > 0 else -1 if size_seq and size_seq[0] < 0 else 0
    first_packet_direction.append(first_direction)

    for i in range(1, len(size_seq)):
        if size_seq[i] * current_direction <= 0:  # 방향이 바뀌면 새 버스트 시작
            bursts += 1
            outgoing_counts.append(sum(1 for size in size_seq[i - current_burst_length:i] if size > 0))
            burst_packet_counts.append(current_burst_length)
            longest_burst = max(longest_burst, current_burst_length)
            current_direction = size_seq[i]
            current_burst_length = 1
        else:
            current_burst_length += 1

    if size_seq:
        outgoing_counts.append(sum(1 for size in size_seq[-current_burst_length:] if size > 0))
        burst_packet_counts.append(current_burst_length)
        longest_burst = max(longest_burst, current_burst_length)

    # Feature 25: Number of Packets in the Longest Burst 저장
    num_packets_longest_burst.append(longest_burst)

In [ ]:
data = {
   'feature 24':first_packet_direction,
   'feature 25':num_packets_longest_burst
}

df = pd.DataFrame(data)
print(df.head())

   feature 24  feature 25
0          -1          32
1          -1         100
2          -1          33
3          -1          85
4          -1          70


In [ ]:
print(len(num_total_packets))
print(len(sum_packets))
print(len(num_incoming_packets))
print(len(frac_incoming_packets))
print(len(num_outgoing_packets))
print(len(frac_outgoing_packets))
print(len(cumulative_packet_size_seq))
print(len(packet_size_ratios))
print(len(burst_lengths))
print(len(burst_seq))
print(len(average_incoming_ordering))
print(len(std_dev_incoming_ordering))
print(len(average_outgoing_ordering))
print(len(std_dev_outgoing_ordering))
print(len(avg_packet_size))
print(len(max_packet_size))
print(len(avg_inter_arrival_time))
print(len(var_packet_size))
print(len(alternative_packets_per_second_sum))
print(len(num_packets_per_second))
print(len(mean_packets_per_second))
print(len(std_packets_per_second))
print(len(med_packets_per_second))
print(len(first_packet_direction))
print(len(num_packets_longest_burst))

3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000
3000


In [ ]:
unmon_data_14 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'cumulative_packet_size_seq': cumulative_packet_size_seq,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'burst_seq': burst_seq,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering
}

df_14 = pd.DataFrame(unmon_data_14)
print(df_14.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0                130        66560                   101   
1               9927      5082624                  9189   
2                359       183808                   285   
3               9948      5093376                  9407   
4               2508      1284096                  2215   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               0.776923                    29               0.223077   
1               0.925657                   738               0.074343   
2               0.793872                    74               0.206128   
3               0.945617                   541               0.054383   
4               0.883174                   293               0.116826   

                          cumulative_packet_size_seq  packet_size_ratios  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...            0.287129   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...  

In [ ]:
unmon_data_19 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'cumulative_packet_size_seq': cumulative_packet_size_seq,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'burst_seq': burst_seq,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
    'avg_packet_size':avg_packet_size,
    'max_packet_size':max_packet_size,
    'avg_inter_arrival_time':avg_inter_arrival_time,
    'var_packet_size':var_packet_size,
    'alternative_packets_per_second_sum':alternative_packets_per_second_sum
}

df_19 = pd.DataFrame(unmon_data_19)
print(df_19.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0                130        66560                   101   
1               9927      5082624                  9189   
2                359       183808                   285   
3               9948      5093376                  9407   
4               2508      1284096                  2215   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               0.776923                    29               0.223077   
1               0.925657                   738               0.074343   
2               0.793872                    74               0.206128   
3               0.945617                   541               0.054383   
4               0.883174                   293               0.116826   

                          cumulative_packet_size_seq  packet_size_ratios  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...            0.287129   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...  

In [ ]:
unmon_data_25 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'cumulative_packet_size_seq': cumulative_packet_size_seq,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'burst_seq': burst_seq,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
    'avg_packet_size':avg_packet_size,
    'max_packet_size':max_packet_size,
    'avg_inter_arrival_time':avg_inter_arrival_time,
    'var_packet_size':var_packet_size,
    'alternative_packets_per_second_sum':alternative_packets_per_second_sum,
    'num_packets_per_second':num_packets_per_second,
    'mean_packets_per_second':mean_packets_per_second,
    'std_packets_per_second':std_packets_per_second,
    'med_packets_per_second':med_packets_per_second,
    'first_packet_direction':first_packet_direction,
    'num_packets_longest_burst':num_packets_longest_burst
}

df_25 = pd.DataFrame(unmon_data_25)
print(df_25.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0                130        66560                   101   
1               9927      5082624                  9189   
2                359       183808                   285   
3               9948      5093376                  9407   
4               2508      1284096                  2215   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               0.776923                    29               0.223077   
1               0.925657                   738               0.074343   
2               0.793872                    74               0.206128   
3               0.945617                   541               0.054383   
4               0.883174                   293               0.116826   

                          cumulative_packet_size_seq  packet_size_ratios  \
0  [-512, -1024, -512, -1024, -512, -1024, -512, ...            0.287129   
1  [-512, -1024, -512, -1024, -512, -1024, -512, ...  

In [ ]:
label = {'label': y}
df_label = pd.DataFrame(label)

print(len(df_label))
print(df_label.head())

3000
   label
0     -1
1     -1
2     -1
3     -1
4     -1


In [ ]:
df_14.to_csv('/content/drive/MyDrive/ML_2024_2/unmon_features14.csv', index=False)
df_19.to_csv('/content/drive/MyDrive/ML_2024_2/unmon_features19.csv', index=False)
df_25.to_csv('/content/drive/MyDrive/ML_2024_2/unmon_features25.csv', index=False)
df_label.to_csv('/content/drive/MyDrive/ML_2024_2/unmon_label.csv', index=False)